## Load in the dataset CSV

## Make one column with classification

This is the subspecies (which may is a mix of two for some specimens, the hybrids); this is for coloring the distribution plot by hybrid status and showing all together.

## Get distribution of images by subspecies (colored by hybrid status)

## Start here

## Import training requirements

In [1]:
import torch
import sys
import os
import csv
from pathlib import Path
# Add your path of library
sys.path.append('../DINO_train')
import training

## Load Training Data

In [2]:
# TODO: Change to the correct path for your .csv file (butterfly_anomaly_train.csv)
training.DATA_FILE = Path('../input_data/butterfly_anomaly_train.csv')
# training.IMG_DIR = Path('./sample_images')
# TODO: Change to the correct path for your images
training.IMG_DIR = Path('../input_data/both')
train_data, test_data = training.load_data(training.DATA_FILE, training.IMG_DIR)
print(train_data.shape)

Filtering bad urls: 0it [00:00, ?it/s]

Filtering bad urls: 4090it [00:02, 1398.78it/s]

Bad rows: 0
(3272, 11)


## Model setup


In [3]:
training.DEVICE='cuda' if torch.cuda.is_available() else 'cpu'
model = training.get_dino_model(dino_name='facebook/dinov2-base').to(training.DEVICE)
training.BATCH_SIZE = 4

# Location to save checkpoints and results
training.CLF_SAVE_DIR = Path('./trained_clfs')
os.makedirs(training.CLF_SAVE_DIR, exist_ok=True)
print(training.DEVICE)
# print(model)
print(type(model))


cpu
<class 'transformers.models.dinov2.modeling_dinov2.Dinov2Model'>


## Start training

In [4]:
# Create dataloader
tr_sig_dloader, test_dl = training.prepare_data_loaders(train_data, test_data)
print(training.data_transforms())

Created base dataset with 3272 samples
Created base dataset with 818 samples
Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
    Lambda()
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)


In [5]:
# Extract visual features from model 
try: 
    tr_features, tr_labels, test_features, test_labels = training.extract_features(tr_sig_dloader, test_dl, model)
    print(tr_features)
except Exception as e:
    print("Error in extracting features")
    print(e)

Extracting features:  34%|███▍      | 278/818 [28:53<37:07,  4.13s/it]  

In [6]:
print(tr_features)

NameError: name 'tr_features' is not defined

In [ ]:
# Train classifier with visual features
csv_output, score_output = training.train_and_evaluate(tr_features, tr_labels, test_features, test_labels)

Training and evaluating svm...
Saved svm classifier to trained_clfs\trained_svm_classifier.pkl
svm: Acc - 0.9863, Hacc - 0.9729, NHacc - 1.0000

          Hybrid-Recall: 0.9976190476190476
          Hybrid-Precision: 0.95662100456621
          f1-Score: 0.9766899766899767
          ROC AUC: 0.9749401770758555
          Accuracy: 0.9755501222493888
          
Training and evaluating sgd...
Saved sgd classifier to trained_clfs\trained_sgd_classifier.pkl
sgd: Acc - 0.9847, Hacc - 0.9759, NHacc - 0.9938

          Hybrid-Recall: 1.0
          Hybrid-Precision: 0.9567198177676538
          f1-Score: 0.9778812572759022
          ROC AUC: 0.9761306532663316
          Accuracy: 0.9767726161369193
          
Training and evaluating knn...
Saved knn classifier to trained_clfs\trained_knn_classifier.pkl
knn: Acc - 0.9908, Hacc - 0.9819, NHacc - 1.0000

          Hybrid-Recall: 0.9976190476190476
          Hybrid-Precision: 0.95662100456621
          f1-Score: 0.9766899766899767
          ROC AUC:

In [16]:
# Save evaluation results
csv_filename = training.CLF_SAVE_DIR / "classifier_evaluation_results.csv"
with open(csv_filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Configuration", "AUC", "Precision", "Recall", "F1-score"])
    writer.writerows(csv_output)

# Save individual scores
scores_filename = training.CLF_SAVE_DIR / "classifier_scores.csv"
with open(scores_filename, mode='w', newline='') as score_file:
    score_writer = csv.writer(score_file)
    score_writer.writerow(["Index", "Score", "True Label"])
    score_writer.writerows(score_output)